# Práctico 2: Recomendación de videojuegos

En este práctico trabajaremos con un subconjunto de datos sobre [videojuegos de Steam](http://cseweb.ucsd.edu/~jmcauley/datasets.html#steam_data). Para facilitar un poco el práctico, se les dará el conjunto de datos previamente procesado. En este mismo notebook mostraremos el proceso de limpieza, para que quede registro del proceso (de todas maneras, por el tamaño de los datos no recomendamos que pierdan tiempo en el proceso salvo que lo consideren útil a fines personales). 

El conjunto de datos se basa en dos partes: lista de juegos (items), y lista de reviews de usuarios sobre distintos juegos. Este último, en su versión original es muy grande, (pesa 1.3GB), por lo que será solo una muestra del mismo sobre la que trabajarán.

A diferencia del conjunto de datos de LastFM utilizados en el [Práctico 1](./practico1.ipynb), en este caso los datos no están particularmente pensados para un sistema de recomendación, por lo que requerirá de un poco más de trabajo general sobre el dataset.

La idea es que, de manera similar al práctico anterior, realicen un sistema de recomendación. A diferencia del práctico anterior, este será un poco más completo y deberán hacer dos sistemas, uno que, dado un nombre de usuario le recomiende una lista de juegos, y otro que dado el título de un juego, recomiende una lista de juegos similares. Además, en este caso se requiere que el segundo sistema (el que recomienda juegos basado en el nombre de un juego en particular) haga uso de la información de contenido (i.e. o bien harán un filtrado basado en contenido o algo híbrido).

## Obtención y limpieza del conjunto de datos

El conjunto de datos originalmente se encuentra en archivos que deberían ser de formato "JSON". Sin embargo, en realidad es un archivo donde cada línea es un objeto de JSON. Hay un problema no obstante y es que las líneas están mal formateadas, dado que no respetan el estándar JSON de utilizar comillas dobles (**"**) y en su lugar utilizan comillas simples (**'**). Afortunadamente, se pueden evaluar como diccionarios de Python, lo cuál permite trabajarlos directamente.

## Conjunto de datos limpio

Para descargar el conjunto de datos que se utilizará en el práctico, basta con ejecutar la siguiente celda.

In [2]:
%%bash

mkdir -p data/steam/
curl -L -o data/steam/games.json.gz https://cs.famaf.unc.edu.ar/\~ccardellino/diplomatura/games.json.gz
curl -L -o data/steam/reviews.json.gz https://cs.famaf.unc.edu.ar/\~ccardellino/diplomatura/reviews.json.gz

Process is interrupted.


## Ejercicio 1: Análisis Exploratorio de Datos

Ya teniendo los datos, podemos cargarlos y empezar con el práctico. Antes que nada vamos a hacer una exploración de los datos. Lo principal a tener en cuenta para este caso es que debemos identificar las variables con las que vamos a trabajar. A diferencia del práctico anterior, este conjunto de datos no está documentado, por lo que la exploración es necesaria para poder entender que cosas van a definir nuestro sistema de recomendación.

In [3]:
import pandas as pd

### Características del conjunto de datos sobre videojuegos

Las características del conjunto de datos de videojuegos tienen la información necesaria para hacer el "vector de contenido" utilizado en el segundo sistema de recomendación. Su tarea es hacer un análisis sobre dicho conjunto de datos y descartar aquella información redundante.

In [ ]:
games = pd.read_json("./data/steam/games.json.gz")
games.head()

In [108]:
# Completar
content_based_df = games.copy(deep=True)

In [109]:
len(content_based_df.columns)

14

In [110]:
content_based_df.isnull().sum()

publisher          8052
genres             3283
app_name              2
title              2050
release_date       2067
tags                163
discount_price    31910
specs               670
price              1377
early_access          0
id                    2
developer          3299
sentiment          7182
metascore         29458
dtype: int64

In [111]:
# Chequemas columnas con NaN en relacion al total
col_nan_map = {}
for col in content_based_df.columns:
    col_nan_map[col] = len(content_based_df[content_based_df[col].isnull()].index) / len(content_based_df.index)
len(col_nan_map)

14

In [112]:
sorted([(k,v) for k, v in col_nan_map.items()], key=lambda x: x[1])

[('early_access', 0.0),
 ('app_name', 6.223743581764431e-05),
 ('id', 6.223743581764431e-05),
 ('tags', 0.005072351019138012),
 ('specs', 0.020849540998910846),
 ('price', 0.04285047456044811),
 ('title', 0.06379337171308543),
 ('release_date', 0.0643223899175354),
 ('genres', 0.10216275089466315),
 ('developer', 0.1026606503812043),
 ('sentiment', 0.22349463202116074),
 ('publisher', 0.250567916601836),
 ('metascore', 0.916695192158083),
 ('discount_price', 0.992998288470515)]

In [113]:
# Descartamos las columnas metascore; discount_price la podemos convertir a dos columnas...
# wih_discount y with_out_discount
content_based_df=content_based_df.drop(['metascore'], 1)

In [114]:
# convertimos discount_price a dos columnas wih_discount y with_out_discount
content_based_df['discount_price'].describe()


count    225.000000
mean      11.930533
std       17.492643
min        0.490000
25%        1.390000
50%        4.190000
75%       22.660000
max      139.990000
Name: discount_price, dtype: float64

In [115]:
import math
content_based_df['with_discount']=content_based_df.apply(lambda x: 1 if x['discount_price'] != math.nan else 0, axis=1)
content_based_df['with_out_discount']=content_based_df.apply(lambda x: 0 if x['discount_price'] != math.nan else 1, axis=1)
content_based_df=content_based_df.drop('discount_price', 1)

In [116]:
# descartamos titulo, app_name, release_date (esto es subjetivo) y sentiment (no es contenido es provisto por el usuario).
content_based_df=content_based_df.drop(['title', 'app_name', 'release_date', 'sentiment'], 1)

In [117]:
# descartamos columna developer
content_based_df=content_based_df.drop('developer', 1)
# decartamos filas con NaN en id y publisher
content_based_df=content_based_df.dropna(subset=['id', 'publisher'])

In [118]:
# a price la descartamos.
content_based_df=content_based_df.drop('price', 1)

In [119]:
# tanto tags, specs y genres tienen valores que se solapan, vamos a quedarnos con valores unicos y a esos hacerles one hot encodig.
def to_genres_tag_specs(x):
    genres_tag_specs = []
    for col in ['genres', 'tags', 'specs']:
        if type(x[col]) is list:
            genres_tag_specs.extend(x[col])
    genres_tag_specs = [x.lower() for x in genres_tag_specs]
    result = set()
    for item in genres_tag_specs:
        result.add(item)
    return sorted(list(result))
content_based_df['genres_tag_specs']=content_based_df.apply(lambda x: to_genres_tag_specs(x), axis=1)

In [120]:
# one hot a la columna tags
content_based_df = content_based_df.join(content_based_df['genres_tag_specs'].str.join('|').str.get_dummies())


In [121]:
content_based_df=content_based_df.drop(['genres', 'tags', 'specs', 'genres_tag_specs'], 1)

In [122]:
# que hacemos con publisher. 
len(content_based_df['publisher'].unique())

8239

In [125]:
# Son 8239 publishers, convendra un embedding?, la vamos a ignorar.

In [126]:
content_based_df=content_based_df.drop('publisher', 1)

KeyError: "['publisher'] not found in axis"

In [21]:
# one hot early access.
content_based_df['with_early_access']=content_based_df.apply(lambda x: 1 if x['early_access'] == True else 0, axis=1)
content_based_df['with_out_early_access']=content_based_df.apply(lambda x: 0 if x['early_access'] == False else 1, axis=1)
content_based_df=content_based_df.drop('early_access', 1)

In [22]:
# chequeamos que no queda nada con valores faltantes.
content_based_df.isnull().sum().sum()

0

In [23]:
content_based_df.head()

,id,with_discount,with_out_discount,1980s,1990's,2.5d,2d,2d fighter,3d platformer,3d vision,...,web publishing,werewolves,western,word game,world war i,world war ii,wrestling,zombies,with_early_access,with_out_early_access
0,761140.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,643980.0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,670290.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,767400.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,772540.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# guardamos un mapa de indice-id
content_based_id_idx = dict(zip(content_based_df['id'], list(content_based_df.index)))
content_based_df=content_based_df.drop('id', 1)

### Características del conjunto de datos de reviews

Este será el conjunto de datos a utilizar para obtener información sobre los usuarios y su interacción con videojuegos. Como se puede observar no hay un rating explícito, sino uno implícito a calcular, que será parte de su trabajo (deberán descubrir que característica les puede dar información que puede ser equivalente a un rating).

In [76]:
reviews = pd.read_json("./data/steam/reviews.json.gz")
reviews.head()

,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id
0,SPejsMan,227940,0,Just one word... Balance!,23.0,92.0,2015-02-25,True,3159,None,NaN,NaN
1,Spodermen,270170,4,Graphics: none\nMusic: Makes me want to sleep\...,4.9,217.0,2014-08-26,False,231,None,NaN,7.656120e+16
2,josh,41700,1,"cheeki breeki iv danke, stalker",53.2,78.0,2015-12-25,False,191,None,NaN,NaN
3,Sammyrism,332310,9,I am really underwhelmed by the small about of...,16.2,178.0,2015-06-04,True,570,None,NaN,NaN
4,moonmirroir,303210,9,"I came into the game expecting nothing, of cou...",1.8,13.0,2015-10-02,False,967,None,NaN,NaN


In [77]:
# Completar
# q es user_id?
print(reviews['user_id'].isnull().sum()/len(reviews['user_id'].index))

0.5920185714285714


In [78]:
# 59% no tiene user_id ... creamo uno propio.

In [79]:
print(reviews['username'].isnull().sum()/len(reviews['username'].index))

0.0


In [80]:
username_unique = reviews.drop_duplicates(subset = ['username'], keep = 'first')['username'].to_dict()
my_id_username_map = {v: k for k, v in username_unique.items()}
reviews = reviews.drop(['user_id'] , axis = 1)    
reviews.insert(0, 'user_id', reviews.username.map(my_id_username_map))

In [81]:
print(reviews['product_id'].isnull().sum()/len(reviews['product_id'].index))

0.0


In [82]:
# NaN en horas, lo usaremos como un rating.
print(reviews['hours'].isnull().sum()/len(reviews['hours'].index))
reviews=reviews.dropna(subset=['hours'])
print(reviews['hours'].isnull().sum()/len(reviews['hours'].index))

0.0034885714285714286
0.0


In [83]:
# remover casos donde un mismo usr dejo mas de una review para un producto.
reviews=reviews.drop_duplicates(subset=['user_id', 'product_id'], keep='first')

In [84]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
hours_norm = min_max_scaler.fit_transform(reviews.hours.values.reshape(-1, 1))
reviews['hours_norm'] = hours_norm

In [85]:
len(reviews.index)

688760

In [98]:
reviews = reviews.sample(n=1000, random_state=47)
reviews.tail()

,user_id,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,hours_norm
76782,76782,SIMPERISMO,319630,8,Nice Game with a very nice Story.,19.1,24.0,2017-08-27,False,412,None,NaN,0.001028
98957,98957,j.toohey,391540,4,BEST GAME EVER,31.2,2.0,2016-06-09,False,2661,None,NaN,0.001680
289361,289361,Civhawk,252490,7,Great game i really thing you should pick it u...,553.0,60.0,2014-01-13,True,9686,None,1.0,0.029778
185338,185338,kijji,240760,7,This is the first CRPG (computer role-playing ...,107.8,163.0,2014-09-25,False,344,None,NaN,0.005805
431782,431782,francescorenCSGO500,273110,2,for the people never played the csgo you will ...,799.3,15.0,2016-06-07,False,331,None,NaN,0.043040


## Ejercicio 2 - Sistema de Recomendación Basado en Usuarios

Este sistema de recomendación deberá entrenar un algoritmo y desarrollar una interfaz que, dado un usuario, le devuelva una lista con los juegos más recomendados.

In [87]:
# Completar
from surprise import Reader
from surprise import Dataset
from surprise import NormalPredictor
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise.accuracy import rmse, mae
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
import numpy

In [88]:
reader = Reader(rating_scale=(numpy.amin(reviews['hours_norm']), 
                              numpy.amax(reviews['hours_norm'])))
data = Dataset.load_from_df(reviews[['user_id', 'product_id', 'hours_norm']], reader)

In [89]:
trainset = data.build_full_trainset()
model = KNNWithZScore(sim_options={'name': 'pearson', 'user_based': True})
model.fit(trainset)
testset = trainset.build_anti_testset()
predictions = model.test(testset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [90]:
def predictions_per_users(predictions):
    pedictions_per_users = {}
    for user_id, item_id, _, est, _ in predictions:
        try:
            pedictions_per_users[user_id].append((item_id, est))
        except:
            pedictions_per_users[user_id] = []
            pedictions_per_users[user_id].append((item_id, est))
    return pedictions_per_users

def best_predictions_per_users(pedictions_per_user, top_n):
    best_pedictions_per_users = {}
    for user_id, user_predicted_ratings in pedictions_per_user.items():
        user_predicted_ratings.sort(key = lambda x: x[1], reverse = True)
        best_pedictions_per_users[user_id] = user_predicted_ratings[:top_n]
    return best_pedictions_per_users
    
best_predictions_per_users = best_predictions_per_users(predictions_per_users(predictions), 5)

In [104]:
def best_predictions_for_user(user_name, best_predictions_per_user):
    user_id = reviews[reviews['username'].str.lower() == user_name.lower()]['user_id'].values[0]
    print(user_name, user_id)
    for product_id, _ in best_predictions_per_user[user_id]:
        print(games[games['id'] == product_id][['title']].values[0])

In [105]:
best_predictions_for_user('kijji', best_predictions_per_users)

kijji 185338
['Vector']
['DARK SOULS™: Prepare To Die™ Edition']
["Jets'n'Guns Gold"]
['Bloodwood Reload']
['Darksiders II Deathinitive Edition']


## Ejercicio 3 - Sistema de Recomendación Basado en Juegos

Similar al caso anterior, con la diferencia de que este sistema espera como entrada el nombre de un juego y devuelve una lista de juegos similares. El sistema deberá estar programado en base a información de contenido de los juegos (i.e. filtrado basado en contenido o sistema híbrido).

In [106]:
# Completar
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(content_based_df, content_based_df)

NameError: name 'content_based_df' is not defined

In [ ]:
def similar_games_by_content(game_title, nb_of_recommendations):
    game_id = games[games['title'].str.lower() == game_title.lower()]['id'].values[0]
    game_idx = dict(zip(games['id'], list(games.index)))
    idx = game_idx[game_id]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:(nb_of_recommendations+1)]
    similar = [i[0] for i in sim_scores]
    return games['title'].iloc[similar].tolist()

In [ ]:
game_title = 'Lost Summoner Kitty'
print(similar_games_by_content(game_title, 5))
